In [ ]:
%%capture

##install required packages
from os import listdir
import os
import glob
import random
import numpy as np
import time
import pickle

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from keras.models import load_model
from gc import callbacks

import cv2 as cv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# If you are using Google Drive, mount it to Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#If using Google Colab Pro or Pro+, enable GPU and high RAM usage
device = 'GPU:0' #'cuda:0'

from psutil import virtual_memory

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print()

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#display image in multiple color space representations
rgb = mpimg.imread('path/to/image/1PCRGAOF7S.jpg')
hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV_FULL)
ycc = cv.cvtColor(rgb, cv.COLOR_RGB2YCrCb)
lab = cv.cvtColor(rgb, cv.COLOR_RGB2Lab)

plt.figure(figsize=(24,24))
components = {"RGB": ["R", "G", "B"], "HSV": ["H", "S", "V"], "YCrCb": ["Y", "Cr", "Cb"], "LAB": ["L", "A", "B"]}

for i in range(16):

  space_nr = i%4


  plt.subplot(4,4,i+1)
  if i < 4:

    space = "RGB"

    if space_nr == 0:
      plt.imshow(rgb[:,:,:])
      plt.axis('off')
      plt.title(f'{space}', fontsize= 30)

    else: 
      plt.imshow(rgb[:,:,space_nr-1])
      plt.axis('off')
      plt.title(f'{components[space][space_nr-1]}', fontsize= 30)

  elif i <8:

      space = "HSV"

      if space_nr == 0:
        plt.imshow(hsv[:,:,:])
        plt.axis('off')
        plt.title(f'{space}', fontsize= 30)

      else: 
        plt.imshow(hsv[:,:,space_nr-1])
        plt.axis('off')
        plt.title(f'{components[space][space_nr-1]}', fontsize= 30)

  elif i <12:

      space = "YCrCb"

      if space_nr == 0:
        plt.imshow(ycc[:,:,:])
        plt.axis('off')
        plt.title(f'{space}', fontsize= 30)

      else: 
        plt.imshow(ycc[:,:,space_nr-1])
        plt.axis('off')
        plt.title(f'{components[space][space_nr-1]}', fontsize= 30)

  else:
      
      space = "LAB"
      
      if space_nr == 0:
        plt.imshow(lab[:,:,:])
        plt.axis('off')
        plt.title(f'{space}', fontsize= 30)

      else: 
        plt.imshow(lab[:,:,space_nr-1])
        plt.axis('off')
        plt.title(f'{components[space][space_nr-1]}', fontsize= 30)

In [ ]:
######### Only run the first time #########


# First, download the the dataset from https://www.kaggle.com/datasets/xhlulu/140k-real-and-fake-faces and https://www.kaggle.com/datasets/hamzaboulahia/hardfakevsrealfaces

#train_ds_paths = glob.glob("/path/train/*/*.jpg") #dataset xhlulu
#val_ds_paths = glob.glob("/path/val/*/*.jpg") #dataset xhlulu
#test_ds_paths = glob.glob("/path/test/*/*.jpg") #dataset xhlulu

#test_ds_paths_3_real = glob.glob("/path/test3/real/*.jpg") #dataset hamzaboulahia
#test_ds_paths_3_fake = glob.glob("/path/test3/fake/*.jpg") #dataset hamzaboulahia


##################
# Google Colab retrieves files faster from Google Drive if they are distributed in more subfolders. Therefore, the train, validation and test data are further divided into multiple subfolders. 

#for dataset_path,name in zip([train_ds_paths, val_ds_paths, test_ds_paths], ['train', 'val', 'test']):
#  for subfolder in range(len(dataset_path)//2500):
#    paths = dataset_path[subfolder*2500: (subfolder+1)*2500]
      
    #os.makedirs(f"/path/{name}/fake/{subfolder}/")
    #os.makedirs(f"/path/{name}/real/{subfolder}/")

#    for path in paths:
#      classification = path.split("/")[7]
#      file_name = path.rsplit("/",1)[1]
#      os.replace(path, f"/path/{name}/{classification}/{subfolder}/{file_name}")

###############
## first remove badly badly cropped or a grey scale images from dataset
## Then, balance the dataset from hamzaboulahia
#if len(test_ds_paths_3_fake)>len(test_ds_paths_3_real):
#  random.shuffle(test_ds_paths_3_fake)
#  for index, path in enumerate(test_ds_paths_3_fake):
#    if index < len(test_ds_paths_3_real):
#      continue
#    else:
#      os.remove(path)


####################
### read the file paths for each set and save them to your directory for fast acces. 

#train_ds_paths = glob.glob("/path/train/*/*/*.jpg")
#val_ds_paths = glob.glob("/path/val/*/*/*.jpg")
#test_ds_paths = glob.glob("/path/test/*/*/*.jpg")
#test_ds_paths_3 = glob.glob("/path/test3/*/*.jpg")

#with open(f'/path/train_ds_paths.pickle', 'wb') as handle:
#  pickle.dump(train_ds_paths, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open(f'/path/val_ds_paths.pickle', 'wb') as handle:
#  pickle.dump(val_ds_paths, handle, protocol=pickle.HIGHEST_PROTOCOL)
#
#with open(f'/path/test_ds_paths.pickle', 'wb') as handle:
#  pickle.dump(test_ds_paths, handle, protocol=pickle.HIGHEST_PROTOCOL) 

#with open(f'/path/test2_ds_paths.pickle', 'wb') as handle:
#  pickle.dump(test_ds_paths_2, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open(f'/path/test3_ds_paths.pickle', 'wb') as handle:
#  pickle.dump(test_ds_paths_3, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
## load paths of images from each data split
with open(f'/path/train_ds_paths.pickle', 'rb') as handle:
  train_ds_paths = pickle.load(handle)

with open(f'/path/val_ds_paths.pickle', 'rb') as handle:
  val_ds_paths = pickle.load(handle)

with open(f'/path/test_ds_paths.pickle', 'rb') as handle:
  test_ds_paths = pickle.load(handle)

with open(f'/path/test2_ds_paths.pickle', 'rb') as handle:
  test2_ds_paths = pickle.load(handle)

with open(f'/path/test3_ds_paths.pickle', 'rb') as handle:
  test3_ds_paths = pickle.load(handle)

In [ ]:
#data loader for training and validation

def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)

    if parts[-3]=="real":
      return 0
    
    else:
      return 1

def process_image_rgb(file_path):
    label = get_label(file_path)
    
    image = tf.io.read_file(file_path)
    rgb_image = tf.image.decode_jpeg(image)
    rgb_image = rgb_image.numpy()

    if rgb_image.shape[1]!= image_size:
      rgb_image = cv.resize(rgb_image, (image_size, image_size))
    rgb_image = np.float32(rgb_image)/255
    
    return np.float32(rgb_image), np.float32(label)


def process_image_hsv(file_path):
    label = get_label(file_path)
    
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image)  
    image = image.numpy()

    hsv_image = cv.cvtColor(image, cv.COLOR_RGB2HSV_FULL)
    if hsv_image.shape[1]!= image_size:
      hsv_image = cv.resize(hsv_image, (image_size, image_size))
    hsv_image = hsv_image/255

    return np.float32(hsv_image), np.float32(label)

def process_image_ycrcb(file_path):
    label = get_label(file_path)
    
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image)  
    image = image.numpy()
    
    ycrcb_image = cv.cvtColor(image, cv.COLOR_RGB2YCrCb)
    if ycrcb_image.shape[1]!= image_size:
      ycrcb_image = cv.resize(ycrcb_image, (image_size, image_size))
    ycrcb_image = ycrcb_image/255

    return np.float32(ycrcb_image), np.float32(label)

def process_image_lab(file_path):
    label = get_label(file_path)
    
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image) 
    image = image.numpy() 
    
    
    lab_image = cv.cvtColor(image, cv.COLOR_RGB2Lab)
    if lab_image.shape[1]!= image_size:
      lab_image = cv.resize(lab_image, (image_size, image_size))
    lab_image = lab_image/255

    return np.float32(lab_image), np.float32(label)

def process_image_combined(file_path):
    label = get_label(file_path)
    
    image = tf.io.read_file(file_path)
    rgb_image = tf.image.decode_jpeg(image) 
    rgb_image = rgb_image.numpy()

    if rgb_image.shape[1]!= image_size:
      rgb_image = cv.resize(rgb_image, (image_size, image_size))

   
    hsv_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2HSV_FULL)
    hsv_image = hsv_image/255    
    
    ycrcb_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2YCrCb)
    ycrcb_image = ycrcb_image/255
    
    lab_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2Lab)
    lab_image = lab_image/255
    
    rgb_image = rgb_image/255

    stacked_image = np.array([np.concatenate(x,axis=1) for x in zip(rgb_image, hsv_image, ycrcb_image, lab_image)])

    return np.float32(stacked_image), np.float32(label)

def set_shape(video, label):

  if color_space == "rgb" or color_space == "hsv" or color_space == "ycrcb" or color_space == "lab":
    video.set_shape((256,256, 3))
    label.set_shape([])

    return video, label
  else:
    video.set_shape((256,256, 12))
    label.set_shape([])

    return video, label

def augment(image, label):
    
    import albumentations as A
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit = 10, p = 0.5)])
    transformed = transform(image=image)
    transformed_image = transformed["image"]


    image = tf.image.random_flip_left_right(image)
    image = Image.Image.rotate(image, degrees)

    return image, label

def data_loader (dataset_paths , color_space):
  AUTOTUNE = tf.data.experimental.AUTOTUNE

  random.shuffle(dataset_paths)

  ds = tf.data.Dataset.from_tensor_slices(dataset_paths)
  ds = ds.shuffle(batch_size*8, reshuffle_each_iteration=True)

  if color_space == "rgb":
      
    ds = ds.map(lambda item: tf.numpy_function(
         process_image_rgb, [item], (tf.float32, tf.float32)) ,num_parallel_calls=AUTOTUNE)  
  
  elif color_space == "hsv":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_hsv, [item], (tf.float32, tf.float32)) ,num_parallel_calls=AUTOTUNE) 

  elif color_space == "ycrcb":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_ycrcb, [item], (tf.float32, tf.float32)) ,num_parallel_calls=AUTOTUNE)     

  elif color_space == "lab":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_lab, [item], (tf.float32, tf.float32)) ,num_parallel_calls=AUTOTUNE) 
    
  else:

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_combined, [item], (tf.float32, tf.float32)) ,num_parallel_calls=AUTOTUNE) 
  
  ds = ds.map(set_shape)
  
  ds = ds.batch(batch_size)
  ds = ds.prefetch(AUTOTUNE)

  return ds


In [ ]:
#data loader for testing

def get_label_test(file_path):
  labels = np.zeros(len(file_path))
  for index, path in enumerate(file_path):
    classification = path.split('/')[7]

    if classification =="real":
      continue
    
    else:
      labels[index]=1

  return labels

def process_image_rgb_test(file_path):    
    image = tf.io.read_file(file_path)
    rgb_image = tf.image.decode_jpeg(image)
    rgb_image = rgb_image.numpy()


    if rgb_image.shape[1]!= image_size:
      rgb_image = cv.resize(rgb_image, (image_size, image_size))
    rgb_image = np.float32(rgb_image)/255
    
    return np.float32(rgb_image)

def process_image_hsv_test(file_path):  
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image)  
    image = image.numpy()

    hsv_image = cv.cvtColor(image, cv.COLOR_RGB2HSV_FULL)
    if hsv_image.shape[1]!= image_size:
      hsv_image = cv.resize(hsv_image, (image_size, image_size))
    hsv_image = hsv_image/255

    return np.float32(hsv_image)

def process_image_ycrcb_test(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image)  
    image = image.numpy()
    
    ycrcb_image = cv.cvtColor(image, cv.COLOR_RGB2YCrCb)
    if ycrcb_image.shape[1]!= image_size:
      ycrcb_image = cv.resize(ycrcb_image, (image_size, image_size))
    ycrcb_image = ycrcb_image/255

    return np.float32(ycrcb_image)

def process_image_lab_test(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image) 
    image = image.numpy() 
    
    lab_image = cv.cvtColor(image, cv.COLOR_RGB2Lab)
    if lab_image.shape[1]!= image_size:
      lab_image = cv.resize(lab_image, (image_size, image_size))
    lab_image = lab_image/255

    return np.float32(lab_image)

def process_image_combined_test(file_path):
    image = tf.io.read_file(file_path)
    rgb_image = tf.image.decode_jpeg(image) 
    rgb_image = rgb_image.numpy()

    if rgb_image.shape[1]!= image_size:
      rgb_image = cv.resize(rgb_image, (image_size, image_size))


    hsv_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2HSV_FULL)
    hsv_image = hsv_image/255    
    
    ycrcb_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2YCrCb)
    ycrcb_image = ycrcb_image/255
    
    lab_image = cv.cvtColor(rgb_image, cv.COLOR_RGB2Lab)
    lab_image = lab_image/255

    rgb_image = rgb_image/255
    
    stacked_image = np.array([np.concatenate(x,axis=1) for x in zip(rgb_image, hsv_image, ycrcb_image, lab_image)])

    return np.float32(stacked_image)

def set_shape_test(video):
  if color_space == "rgb" or color_space == "hsv" or color_space == "ycrcb" or color_space == "lab":
    video.set_shape((256,256, 3))
    return video
  else:
    video.set_shape((256,256, 12))
    return video



def data_loader_test (dataset_paths , color_space):
  AUTOTUNE = tf.data.experimental.AUTOTUNE

  ds = tf.data.Dataset.from_tensor_slices(dataset_paths)

  if color_space == "rgb":
      
    ds = ds.map(lambda item: tf.numpy_function(
         process_image_rgb_test, [item], tf.float32) ,num_parallel_calls=AUTOTUNE) 
      
  elif color_space == "hsv":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_hsv_test, [item], tf.float32) ,num_parallel_calls=AUTOTUNE) 

  elif color_space == "ycrcb":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_ycrcb_test, [item], tf.float32) ,num_parallel_calls=AUTOTUNE)     

  elif color_space == "lab":

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_lab_test, [item], tf.float32) ,num_parallel_calls=AUTOTUNE) 
    
  else:

    ds = ds.map(lambda item: tf.numpy_function(
         process_image_combined_test, [item], tf.float32) ,num_parallel_calls=AUTOTUNE) 
  
  ds = ds.map(set_shape_test)

  ds = ds.batch(batch_size)
  ds = ds.prefetch(AUTOTUNE)
  #ds = ds.cache()

  return ds


In [ ]:
#initialize data loader

tf.random.set_seed(2022)

image_size = 256
batch_size = 16
color_space = 'rgb'
channels = 3

train_ds =  data_loader(train_ds_paths, color_space)
val_ds =  data_loader(val_ds_paths, color_space)

test1_ds =  data_loader_test(test_ds_paths, color_space)
test2_ds =  data_loader_test(test2_ds_paths, color_space)
test3_ds =  data_loader_test(test3_ds_paths, color_space)

test1_labels =  get_label_test(test_ds_paths)
test2_labels =  get_label_test(test2_ds_paths)
test3_labels =  get_label_test(test3_ds_paths)


for image, label in train_ds.take(3):
  print(image.shape, label)

for image, label in val_ds.take(3):
  print(image.shape, label)

for image in test1_ds.take(3):
  print(image.shape)

for image in test3_ds.take(3):
  print(image.shape)

In [ ]:
def create_CNN_model():
    #initiate model
    
    model = Sequential()
    
    model.add(Conv2D(512, (3, 3), padding='same',activation = 'relu'))
    
    model.add(MaxPooling2D((4, 4), padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(512, (3, 3), padding='same',activation = 'relu'))
    model.add(MaxPooling2D((2, 2),padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(512, (3, 3), padding='same',activation = 'relu'))
    model.add(MaxPooling2D((2, 2),padding = "same"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(512, (3, 3), padding='same',activation = 'relu'))
    model.add(MaxPooling2D((2, 2,), padding = "same"))
    model.add(BatchNormalization())

    model.add(Flatten())
    
    model.add(layers.Dense(512, activation= 'relu'))
    model.add(Dropout(0.3))
    model.add(layers.BatchNormalization())
    model.add(layers.Dense(512, activation= 'relu'))
    model.add(Dropout(0.3))
    model.add(layers.Dense(512, activation= 'relu'))
    model.add(Dropout(0.3))
    model.add(layers.Dense(1, activation= 'sigmoid'))
  
    # Display the models summary.
    model.summary()
    
    return model

In [ ]:
# Construct the CNN
CNN_model = create_CNN_model()

# Add callbacks 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir= f'/path/history/CNN-{color_space}-history', histogram_freq=1,  write_graph=True, write_images=True)

#Add early stopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 2, mode = 'min', restore_best_weights = True)

save_path = "/path/models/CNN-rgb-{epoch:02d}-{val_loss:.2f}.keras"
save_model = ModelCheckpoint(save_path, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

 
# Compile the model 
CNN_model.compile(loss='binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

configproto = tf.compat.v1.ConfigProto() 
configproto.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=configproto) 
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
#Train model

#CNN_model = load_model('/path/models/CNN-ycrcb-06-0.31.keras')

CNN_model_training_history = CNN_model.fit(train_ds,  validation_data= val_ds, epochs = 10,   callbacks = [early_stopping_callback, save_model]) 


In [ ]:
%%capture
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers

import tensorflow_addons as tfa

In [ ]:
# code based on https://github.com/keras-team/keras-io/blob/master/examples/vision/image_classification_with_vision_transformer.py
num_classes = 2
input_shape = (256, 256, channels)

learning_rate = 0.001
weight_decay = 0.0001

num_epochs = 50
image_size = 256 
patch_size = 16  
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 8
transformer_units = [
    projection_dim * 2,
    projection_dim,] 
transformer_layers = 8
mlp_head_units = [2048, 2048]  

#Implement multilayer perceptron (MLP)
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

#Implement patch creation as a layer
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

#Implement the patch encoding layer
#The PatchEncoder layer will linearly transform a patch by projecting it into a vector of size projection_dim. In addition, it adds a learnable position embedding to the projected vector.

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    #logits = layers.Dense(num_classes)(features)
    outputs = layers.Dense(1, activation= 'sigmoid')(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)#logits
    
    model.summary()

    return model

In [ ]:
# Display image patches for the ViT
plt.figure(figsize=(4, 4))

image=mpimg.imread('/path/1PCRGAOF7S.jpg')

plt.imshow(image.astype("uint8"))
plt.axis("off")

resized_image = tf.image.resize(
    tf.convert_to_tensor([image]), size=(256, 256)
)
patches = Patches(patch_size)(resized_image)
print(f"Image size: {image_size} X {image_size}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = int(np.sqrt(patches.shape[1]))
plt.figure(figsize=(4, 4))
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, 3))
    plt.imshow(patch_img.numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
# Construct the ViT
vit_classifier = create_vit_classifier()

#construct optimizer
optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay)

#compile model
vit_classifier.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=["accuracy"])

#add callback
checkpoint_filepath = "/path/models/vit-ycrcb/vit2-ycrcb-{epoch:02d}-{val_loss:.2f}.keras"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=False,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch')

#vit_classifier.load_weights("/path/models/vit-ycrcb/vit-ycrcb-20-0.20.keras")

# train model
history = vit_classifier.fit(train_ds,
    batch_size=batch_size,
    epochs=10,
    validation_data= val_ds,
    callbacks=[checkpoint_callback],)

In [ ]:
epochs10 = [1,2,3,4,5,6,7,8,9,10]
epochs20 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
epochs30 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]

#load models for testing
cnn_models_rgb = glob.glob('/path/models/cnn-rgb/*.keras')
cnn_models_rgb = sorted(cnn_models_rgb, key=lambda x: int(x.split('-')[3]), reverse=False)
print(cnn_models_rgb)

cnn_models_hsv = glob.glob('/path/models/cnn-hsv/*.keras')
cnn_models_hsv = sorted(cnn_models_hsv, key=lambda x: int(x.split('-')[3]), reverse=False)
print(cnn_models_hsv)

cnn_models_ycrcb = glob.glob('/path/models/cnn-ycrcb/*.keras')
cnn_models_ycrcb = sorted(cnn_models_ycrcb, key=lambda x: int(x.split('-')[3]), reverse=False)
print(cnn_models_ycrcb)

cnn_models_lab = glob.glob('/path/models/cnn-lab/*.keras')
cnn_models_lab = sorted(cnn_models_lab, key=lambda x: int(x.split('-')[3]), reverse=False)
print(cnn_models_lab)

cnn_models_stacked = glob.glob('/path/models/cnn-stacked/*.keras')
cnn_models_stacked = sorted(cnn_models_stacked, key=lambda x: int(x.split('-')[3]), reverse=False)
print(cnn_models_stacked)


vit_models_rgb = glob.glob('/path/models/vit-rgb/*.keras')
vit_models_rgb = sorted(vit_models_rgb, key=lambda x: int(x.split('-')[3]), reverse=False)
print(vit_models_rgb)


vit_models_hsv = glob.glob('/path/models/vit-hsv/*.keras')
vit_models_hsv = sorted(vit_models_hsv, key=lambda x: int(x.split('-')[3]), reverse=False)
print(vit_models_hsv)

vit_models_ycrcb = glob.glob('/path/models/vit-ycrcb/*.keras')
vit_models_ycrcb = sorted(vit_models_ycrcb, key=lambda x: int(x.split('-')[3]), reverse=False)
print(vit_models_ycrcb)

vit_models_lab = glob.glob('/path/models/vit-lab/*.keras')
vit_models_lab = sorted(vit_models_lab, key=lambda x: int(x.split('-')[3]), reverse=False)
print(vit_models_lab)

vit_models_stacked = glob.glob('/path/models/vit-stacked/*.keras')
vit_models_stacked = sorted(vit_models_stacked, key=lambda x: int(x.split('-')[3]), reverse=False)
print(vit_models_stacked)

#scores gathered by cell below
cnn_models_rgb_acc    =   [0.787, 0.868, 0.924, 0.924, 0.954, 0.937, 0.963, 0.932, 0.972, 0.961]
cnn_models_hsv_acc    =   [0.728, 0.814, 0.856, 0.889, 0.888, 0.910, 0.916, 0.930, 0.940, 0.923]      
cnn_models_ycbcr_acc  =   [0.794, 0.834, 0.885, 0.859, 0.955, 0.951, 0.968, 0.970, 0.965, 0.970]        
cnn_models_lab_acc    =   [0.744, 0.832, 0.879, 0.920, 0.947, 0.951 ,0.908, 0.906, 0.902, 0.937]

cnn_models_rgb_loss   =   [0.506,0.316,0.239,0.187,0.126,0.403,0.297,0.170,0.152,0.273]
cnn_models_hsv_loss   =   [0.546,0.410,0.345,0.256,0.265,0.265,0.299,0.491,0.994,0.751] 
cnn_models_ycbcr_loss =   [0.439,0.382,0.269,0.473,0.149,0.311,0.183,0.131,0.516,0.239]  
cnn_models_lab_loss   =   [0.633,0.387,0.482,0.232,0.359,0.136,0.465,1.063,0.593,0.656] 

cnn_models_rgb_test1  =   [0.788, 0.872, 0.925, 0.924, 0.954, 0.939, 0.963, 0.932, 0.973, 0.964]
cnn_models_hsv_test1  =   [0.733, 0.822, 0.860, 0.891, 0.894, 0.915, 0.919, 0.933, 0.942, 0.920] 
cnn_models_ycbcr_test1=   [0.798, 0.832, 0.886, 0.853, 0.955, 0.953, 0.968, 0.973, 0.965, 0.970]
cnn_models_lab_test1  =   [0.743, 0.833, 0.881, 0.918, 0.946, 0.951, 0.907, 0.904, 0.903, 0.940]  

vit_models_rgb_acc =  [0.500, 0.732, 0.799, 0.838, 0.867, 0.856, 0.903, 0.900, 0.901, 0.913, 0.925, 0.937, 0.924, 0.930, 0.935, 0.941, 0.910, 0.942, 0.945, 0.914]
vit_models_hsv_acc =   [0.629, 0.778, 0.829, 0.853, 0.873, 0.889, 0.825, 0.901, 0.913, 0.9, 0.921, 0.887, 0.927, 0.897, 0.919, 0.93, 0.922, 0.922, 0.936, 0.925]       
vit_models_ycbcr_acc = [0.5, 0.697, 0.766, 0.796, 0.805, 0.842, 0.819, 0.862, 0.875, 0.836, 0.858, 0.895, 0.868, 0.88, 0.876, 0.901, 0.857, 0.9, 0.914, 0.921, 0.926, 0.921, 0.869, 0.932, 0.925, 0.925, 0.928, 0.932, 0.922, 0.93]       
vit_models_lab_acc =   [0.5, 0.651, 0.74, 0.768, 0.805, 0.795, 0.84, 0.862, 0.871, 0.891, 0.888, 0.892, 0.912, 0.9, 0.903, 0.914, 0.91, 0.899, 0.931, 0.92, 0.886, 0.924, 0.916, 0.917, 0.933, 0.931, 0.903, 0.937, 0.938, 0.919]         

vit_models_rgb_loss =   [0.693, 0.536, 0.431, 0.362, 0.329, 0.32, 0.233, 0.238, 0.238, 0.214, 0.201, 0.164, 0.192, 0.184, 0.166, 0.152, 0.222, 0.15, 0.145, 0.21]
vit_models_hsv_loss =   [0.649, 0.461, 0.381, 0.338, 0.298, 0.266, 0.378, 0.238, 0.211, 0.241, 0.205, 0.268, 0.186, 0.241, 0.198, 0.17, 0.192, 0.189, 0.163, 0.187]       
vit_models_ycbcr_loss =  [0.693, 0.595, 0.494, 0.44, 0.419, 0.36, 0.401, 0.316, 0.302, 0.366, 0.333, 0.26, 0.307, 0.286, 0.288, 0.244, 0.323, 0.24, 0.212, 0.195, 0.185, 0.19, 0.301, 0.178, 0.186, 0.191, 0.175, 0.168, 0.188, 0.179]     
vit_models_lab_loss =   [0.694, 0.624, 0.527, 0.48, 0.419, 0.511, 0.364, 0.322, 0.307, 0.262, 0.268, 0.264, 0.221, 0.242, 0.236, 0.212, 0.229, 0.242, 0.175, 0.196, 0.268, 0.192, 0.216, 0.205, 0.177, 0.183, 0.24, 0.161, 0.157, 0.193]   

vit_models_rgb_test1  =   [0.5, 0.737, 0.804, 0.84, 0.866, 0.858, 0.905, 0.904, 0.905, 0.913, 0.923, 0.936, 0.924, 0.927, 0.935, 0.943, 0.910, 0.941, 0.943, 0.913] 
vit_models_hsv_test1 =    [0.627, 0.783, 0.83, 0.854, 0.871, 0.887, 0.829, 0.897, 0.915, 0.901, 0.92, 0.889, 0.927, 0.898, 0.92, 0.929, 0.923, 0.925, 0.935, 0.927] 
vit_models_ycrcb_test1 = 	[0.5, 0.695, 0.767, 0.797, 0.805, 0.839, 0.819, 0.863, 0.874, 0.832, 0.858, 0.891, 0.865, 0.878, 0.874, 0.902, 0.86, 0.897, 0.913, 0.922, 0.927, 0.923, 0.869, 0.93, 0.928, 0.925, 0.93, 0.934, 0.92, 0.932] 
vit_models_lab_test1  =   [0.5, 0.655, 0.741, 0.77, 0.805, 0.788, 0.837, 0.86, 0.869, 0.888, 0.884, 0.891, 0.912, 0.902, 0.899, 0.912, 0.907, 0.897, 0.93, 0.925, 0.886, 0.923, 0.918, 0.915, 0.931, 0.934, 0.907, 0.939, 0.937, 0.921]


In [ ]:
# get validation accuracy and loss
acc_val = []
loss_val = []
for model in cnn_models_rgb:
  #vit_classifier.load_weights(model)
  #acc = vit_classifier.evaluate(data_loader(test1_ds_paths, color_space))

  CNN_model = load_model(model)
  acc = CNN_model.evaluate(data_loader(val_ds_paths, color_space))

  acc_val.append(round(acc[1],3))
  loss_val.append(round(acc[0],3))

print(loss_val)
print(acc_val)

In [ ]:
#plot validation loss CNN
plt.figure(figsize=(4,3), dpi =1200)
plt.plot(epochs10[:5],cnn_models_rgb_loss[:5], 'k-' )
plt.plot(epochs10[:6],cnn_models_hsv_loss[:6], 'k--')
plt.plot(epochs10[:8],cnn_models_ycbcr_loss[:8], 'k-.' )
plt.plot(epochs10[:6],cnn_models_lab_loss[:6], 'k:' )

plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['RGB', 'HSV', 'YCbCr', "LAB"], loc='upper right')
plt.show()

In [ ]:
#plot validation loss ViT
plt.figure(figsize=(4,3), dpi =1200)
plt.plot(epochs20[:19],vit_models_rgb_loss[:19],  'k-' )
plt.plot(epochs20[:19],vit_models_hsv_loss[:19],  'k--' )
plt.plot(epochs30[:24],vit_models_ycbcr_loss[:24],  'k-.' )
plt.plot(epochs30[:25],vit_models_lab_loss[:25],  'k:' )

plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['RGB' , 'HSV', 'YCbCr', 'LAB'], loc='upper right')
plt.show()

In [ ]:
##plot validation accuracy CNN
plt.figure(figsize=(4,3), dpi =1200)

plt.plot(epochs10[:5],cnn_models_rgb_acc[:5], 'k-' )
plt.plot(epochs10[:6],cnn_models_hsv_acc[:6], 'k--')
plt.plot(epochs10[:8],cnn_models_ycbcr_acc[:8], 'k-.' )
plt.plot(epochs10[:6],cnn_models_lab_acc[:6], 'k:' )

plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['RGB' , 'HSV', 'YCbCr', 'LAB'], loc='lower right')
plt.show()

In [ ]:
## plot validation accuracy ViT
plt.figure(figsize=(4,3), dpi =1200)

plt.plot(epochs20[:19],vit_models_rgb_acc[:19],  'k-' )
plt.plot(epochs20[:19],vit_models_hsv_acc[:19],  'k--' )
plt.plot(epochs30[:24],vit_models_ycbcr_acc[:24],  'k-.' )
plt.plot(epochs30[:25],vit_models_lab_acc[:25],  'k:' )

plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['RGB' , 'HSV', 'YCbCr', 'LAB'], loc='lower right')

plt.show()

In [ ]:
#get labels test sets
test1_labels =  get_label_test(test1_ds_paths)
test3_labels =  get_label_test(test3_ds_paths)

In [ ]:
#get test scores cnn
test_scores_cnn = {}
cnn_model =[cnn_models_rgb[4], cnn_models_hsv[5], cnn_models_ycrcb[7], cnn_models_lab[5]]
cnn_color_spaces =['rgb', 'hsv', 'ycrcb', 'lab']
for index, (model, color_space) in enumerate(zip(cnn_model,cnn_color_spaces )):
  print(model)
  print(color_space)

  color_space = color_space
  batch_size = 16
  image_size = 256

  
  CNN_model = load_model(model)

  test1_ds =  data_loader_test(test1_ds_paths, color_space)
  test3_ds =  data_loader_test(test3_ds_paths, color_space)

  test1_labels_pred = CNN_model.predict(test1_ds)
  test3_labels_pred = CNN_model.predict(test3_ds)

  test_scores_cnn[model]= {}

  for j, (true, pred) in enumerate(zip([test1_labels, test3_labels],[test1_labels_pred, test3_labels_pred])):


    pred = np.asarray(pred)
    pred = np.round(pred)
    true = np.asarray(true) 

    report = classification_report(true, pred, target_names=['Real', 'Fake'], output_dict = True)
    acc = round(report[ 'accuracy'],3)
    precision =  round(report['Fake']['precision'],3)
    recall =  round(report['Fake']['recall'],3)
    metrics = [acc, precision, recall]
    test_scores_cnn[model][j] = metrics
  
  print(test_scores_cnn[model])

print(test_scores_cnn)



In [ ]:
#Test results CNN
CRGB = {0: [0.954, 0.950, 0.959], 1: [0.734, 0.990, 0.472]}
CHSV = {0: [0.915, 0.891, 0.945], 1: [0.791, 0.983, 0.593]}
CYCbCr={0: [0.973, 0.962, 0.985], 1: [0.644, 0.908, 0.320]}
CLAB = {0: [0.951, 0.948, 0.955], 1: [0.644, 0.983, 0.292]}

avg_acc_cnn1 = CRGB[0][0] + CHSV[0][0] + CYCbCr[0][0] + CLAB[0][0]
avg_prec_cnn1 = CRGB[0][1] + CHSV[0][1] + CYCbCr[0][1] + CLAB[0][1]
avg_rec_cnn1 = CRGB[0][2] + CHSV[0][2] + CYCbCr[0][2] + CLAB[0][2]

avg_acc_cnn2 = CRGB[1][0] + CHSV[1][0] + CYCbCr[1][0] + CLAB[1][0]
avg_prec_cnn2 = CRGB[1][1] + CHSV[1][1] + CYCbCr[1][1] + CLAB[1][1]
avg_rec_cnn2 = CRGB[1][2] + CHSV[1][2] + CYCbCr[1][2] + CLAB[1][2]

print(round(avg_acc_cnn1/4,3), round(avg_prec_cnn1/4,3), round(avg_rec_cnn1/4,3) )
print(round(avg_acc_cnn2/4,3), round(avg_prec_cnn2/4,3), round(avg_rec_cnn2/4,3) )

In [ ]:
#get test scores ViT

test_scores_vit = {}
vit_model =[vit_models_rgb[18], vit_models_hsv[18], vit_models_ycrcb[23], vit_models_lab[24]]
vit_color_spaces =['rgb', 'hsv', 'ycrcb', 'lab']

for index, (model, color_space) in enumerate(zip(vit_model, vit_color_spaces)):
  print(model)
  print(color_space)
  batch_size = 16
  color_space = color_space
  image_size = 256

  
  vit_classifier.load_weights(model)

  test1_ds =  data_loader_test(test1_ds_paths, color_space)
  test3_ds =  data_loader_test(test3_ds_paths, color_space)

  test1_labels_pred = vit_classifier.predict(test1_ds)
  test3_labels_pred = vit_classifier.predict(test3_ds)

  test_scores_vit[model]= {}

  for j, (true, pred) in enumerate(zip([test1_labels, test3_labels],[test1_labels_pred, test3_labels_pred])):


    pred = np.asarray(pred)
    pred = np.round(pred)
    true = np.asarray(true) 

    report = classification_report(true, pred, target_names=['Real', 'Fake'], output_dict = True)
    acc = round(report[ 'accuracy'],3)
    precision =  round(report['Fake']['precision'],3)
    recall =  round(report['Fake']['recall'],3)
    metrics = [acc, precision, recall]
    test_scores_vit[model][j] = metrics
  
  print(test_scores_vit[model])

print(test_scores_vit)

In [ ]:
#Test results ViT
VRGB = {0: [0.943, 0.946, 0.941], 1: [0.759, 1.0, 0.517]}
VHSV = {0: [0.935, 0.954, 0.914], 1: [0.614, 1.0, 0.228]}
VYCbCr ={0: [0.930, 0.925, 0.936], 1: [0.705, 1.0, 0.41]}
VLAB =  {0: [0.931, 0.924, 0.940], 1: [0.710, 1.0, 0.42]}

avg_acc_vit1 = VRGB[0][0] + VHSV[0][0] + VYCbCr[0][0] + VLAB[0][0]
avg_prec_vit1 = VRGB[0][1] + VHSV[0][1] + VYCbCr[0][1] + VLAB[0][1]
avg_rec_vit1 = VRGB[0][2] + VHSV[0][2] + VYCbCr[0][2] + VLAB[0][2]

avg_acc_vit2 = VRGB[1][0] + VHSV[1][0] + VYCbCr[1][0] + VLAB[1][0]
avg_prec_vit2 = VRGB[1][1] + VHSV[1][1] + VYCbCr[1][1] + VLAB[1][1]
avg_rec_vit2 = VRGB[1][2] + VHSV[1][2] + VYCbCr[1][2] + VLAB[1][2]

print(round(avg_acc_vit1/4,3), round(avg_prec_vit1/4,3), round(avg_rec_vit1/4,3) )
print(round(avg_acc_vit2/4,3), round(avg_prec_vit2/4,3), round(avg_rec_vit2/4,3) )

In [ ]:
#calculate average accuracy, precision and recall
avg_acc_rgb1 = CRGB[0][0] + VRGB[0][0]
avg_acc_hsv1 = CHSV[0][0] + VHSV[0][0]
avg_acc_ycbcr1 = CYCbCr[0][0] + VYCbCr[0][0]
avg_acc_lab1 = CLAB[0][0] + VLAB[0][0]

avg_acc_rgb2 = CRGB[1][0] + VRGB[1][0]
avg_acc_hsv2 = CHSV[1][0] + VHSV[1][0]
avg_acc_ycbcr2 = CYCbCr[1][0] + VYCbCr[1][0]
avg_acc_lab2 = CLAB[1][0] + VLAB[1][0]

avg_prec_rgb1 = CRGB[0][1] + VRGB[0][1]
avg_prec_hsv1 = CHSV[0][1] + VHSV[0][1]
avg_prec_ycbcr1 = CYCbCr[0][1] + VYCbCr[0][1]
avg_prec_lab1 = CLAB[0][1] + VLAB[0][1]

avg_prec_rgb2 = CRGB[1][1] + VRGB[1][1]
avg_prec_hsv2 = CHSV[1][1] + VHSV[1][1]
avg_prec_ycbcr2 = CYCbCr[1][1] + VYCbCr[1][1]
avg_prec_lab2 = CLAB[1][1] + VLAB[1][1]

avg_rec_rgb1 = CRGB[0][2] + VRGB[0][2]
avg_rec_hsv1 = CHSV[0][2] + VHSV[0][2]
avg_rec_ycbcr1 = CYCbCr[0][2] + VYCbCr[0][2]
avg_rec_lab1 = CLAB[0][2] + VLAB[0][2]

avg_rec_rgb2 = CRGB[1][2] + VRGB[1][2]
avg_rec_hsv2 = CHSV[1][2] + VHSV[1][2]
avg_rec_ycbcr2 = CYCbCr[1][2] + VYCbCr[1][2]
avg_rec_lab2 = CLAB[1][2] + VLAB[1][2]

print(round(avg_acc_rgb1/2,3), round(avg_prec_rgb1/2,3), round(avg_rec_rgb1/2,3) )
print(round(avg_acc_rgb2/2,3), round(avg_prec_rgb2/2,3), round(avg_rec_rgb2/2,3) )
print()
print(round(avg_acc_hsv1/2,3), round(avg_prec_hsv1/2,3), round(avg_rec_hsv1/2,3) )
print(round(avg_acc_hsv2/2,3), round(avg_prec_hsv2/2,3), round(avg_rec_hsv2/2,3) )
print()
print(round(avg_acc_ycbcr1/2,3), round(avg_prec_ycbcr1/2,3), round(avg_rec_ycbcr1/2,3) )
print(round(avg_acc_ycbcr2/2,3), round(avg_prec_ycbcr2/2,3), round(avg_rec_ycbcr2/2,3) )
print()
print(round(avg_acc_lab1/2,3), round(avg_prec_lab1/2,3), round(avg_rec_lab1/2,3) )
print(round(avg_acc_lab2/2,3), round(avg_prec_lab2/2,3), round(avg_rec_lab2/2,3) )

In [ ]:
#calculate average accuracy scores of models
average_acc1 = ((round(avg_acc_vit1/4,3) + round(avg_acc_cnn1/4,3))/2) 
average_acc2 = ((round(avg_acc_vit2/4,3) + round(avg_acc_cnn2/4,3))/2)
average_drop_acc_models = ((round(avg_acc_vit1/4,3) + round(avg_acc_cnn1/4,3))/2) - ((round(avg_acc_vit2/4,3) + round(avg_acc_cnn2/4,3))/2)
print("average acc 1",average_acc1 )
print("average acc 2",average_acc2 )
print("average drop acc models", round(average_drop_acc_models,3))

average_drop_recall_models = ((round(avg_prec_vit1/4,3) + round(avg_prec_cnn1/4,3))/2) - ((round(avg_prec_vit2/4,3) + round(avg_prec_cnn2/4,3))/2)
print("average incr prec models", -1*round(average_drop_recall_models,3))

average_drop_recall_models = ((round(avg_rec_vit1/4,3) + round(avg_rec_cnn1/4,3))/2) - ((round(avg_rec_vit2/4,3) + round(avg_rec_cnn2/4,3))/2)
print("average drop rec models", round(average_drop_recall_models,3))

In [ ]:
#calculate variance models
variance_acc_cnn1 = round(np.var(np.asarray([CRGB[0][0] , CHSV[0][0] , CYCbCr[0][0] , CLAB[0][0]])),6)
variance_prec_cnn1 = round(np.var(np.asarray([CRGB[0][1] , CHSV[0][1] , CYCbCr[0][1] , CLAB[0][1]])),6)
variance_rec_cnn1 = round(np.var(np.asarray([CRGB[0][2] , CHSV[0][2] , CYCbCr[0][2] , CLAB[0][2]])),6)

variance_acc_cnn2 = round(np.var(np.asarray([CRGB[1][0] , CHSV[1][0] , CYCbCr[1][0] , CLAB[1][0]])),6)
variance_prec_cnn2 = round(np.var(np.asarray([CRGB[1][1] , CHSV[1][1] , CYCbCr[1][1] , CLAB[1][1]])),6)
variance_rec_cnn2 = round(np.var(np.asarray([CRGB[1][2] , CHSV[1][2] , CYCbCr[1][2] , CLAB[1][2]])),6)

variance_acc_vit1 = round(np.var(np.asarray([VRGB[0][0] , VHSV[0][0] , VYCbCr[0][0] , VLAB[0][0]])),6)
variance_prec_vit1 = round(np.var(np.asarray([VRGB[0][1] , VHSV[0][1] , VYCbCr[0][1] , VLAB[0][1]])),6)
variance_rec_vit1 = round(np.var(np.asarray([VRGB[0][2] , VHSV[0][2] , VYCbCr[0][2] , VLAB[0][2]])),6)

variance_acc_vit2 = round(np.var(np.asarray([VRGB[1][0] , VHSV[1][0] , VYCbCr[1][0] , VLAB[1][0]])),6)
variance_prec_vit2 = round(np.var(np.asarray([VRGB[1][1] , VHSV[1][1] , VYCbCr[1][1] , VLAB[1][1]])),6)
variance_rec_vit2 = round(np.var(np.asarray([VRGB[1][2] , VHSV[1][2] , VYCbCr[1][2] , VLAB[1][2]])),6)

print("variance cnn1", variance_acc_cnn1 , variance_prec_cnn1, variance_rec_cnn1 )
print("variance vit1", variance_acc_vit1 , variance_prec_vit1, variance_rec_vit1 )

print()
print("variance cnn2 ",variance_acc_cnn1 , variance_prec_cnn1, variance_rec_cnn1 )
print("variance vit2 ",variance_acc_vit2 , variance_prec_vit2, variance_rec_vit2 )

In [ ]:
#calculate and save ROC & AUC scores for CNN
cnn_model =[cnn_models_rgb[4], cnn_models_hsv[5], cnn_models_ycrcb[7], cnn_models_lab[5]]
cnn_color_spaces =['rgb', 'hsv', 'ycrcb', 'lab']

def roc_auc_cnn (models, spaces):
  roc_auc= {}

  for index, (model, color_space) in enumerate(zip(models,spaces )):
    print(model)
    print(color_space)
    batch_size = 16
    color_space = color_space
    image_size = 256
    
    CNN_model = load_model(model)

    test1_ds =  data_loader_test(test1_ds_paths, color_space)
    test3_ds =  data_loader_test(test3_ds_paths, color_space)

    test1_labels_pred = CNN_model.predict(test1_ds)
    test3_labels_pred = CNN_model.predict(test3_ds)

    roc_auc[index]= {}

    for j, (true, pred) in enumerate(zip([test1_labels, test3_labels],[test1_labels_pred, test3_labels_pred])):


      roc_auc[index][j] = []

      pred = np.asarray(pred)
      true = np.asarray(true) 

      fpr_keras, tpr_keras, thresholds_keras = roc_curve(true, pred)
      auc_keras = auc(fpr_keras, tpr_keras)

      metrics = [fpr_keras, tpr_keras, auc_keras]
      roc_auc[index][j] = metrics

  return roc_auc

roc_auc_cnn_results = roc_auc_cnn(cnn_model, cnn_color_spaces)

with open(f'/content/gdrive/My Drive/thesis_images/auc/roc_auc_cnn_results.pickle', 'wb') as handle:
  pickle.dump(roc_auc_cnn_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#calculate and save ROC & AUC scores for ViT

vit_model =[vit_models_rgb[18], vit_models_hsv[18], vit_models_ycrcb[23], vit_models_lab[24]]
vit_color_spaces =['rgb', 'hsv', 'ycrcb', 'lab']

def roc_auc_vit (models, spaces):
  roc_auc= {}

  for index, (model, color_space) in enumerate(zip(models,spaces )):
    print(model)
    print(color_space)
    batch_size = 16
    color_space = color_space
    image_size = 256
    
    vit_classifier.load_weights(model)

    test1_ds =  data_loader_test(test1_ds_paths, color_space)
    test3_ds =  data_loader_test(test3_ds_paths, color_space)

    test1_labels_pred = vit_classifier.predict(test1_ds)
    test3_labels_pred = vit_classifier.predict(test3_ds)

    roc_auc[index]= {}

    for j, (true, pred) in enumerate(zip([test1_labels, test3_labels],[test1_labels_pred, test3_labels_pred])):


      roc_auc[index][j] = []

      pred = np.asarray(pred)
      true = np.asarray(true) 

      fpr_keras, tpr_keras, thresholds_keras = roc_curve(true, pred)
      auc_keras = auc(fpr_keras, tpr_keras)

      metrics = [fpr_keras, tpr_keras, auc_keras]
      roc_auc[index][j] = metrics

  return roc_auc

roc_auc_vit_results = roc_auc_vit(vit_model, vit_color_spaces)

with open(f'/content/gdrive/My Drive/thesis_images/auc/roc_auc_vit_results.pickle', 'wb') as handle:
  pickle.dump(roc_auc_vit_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#plot ROC curves and AUCs of CNN and Vit - first test set
with open(f'/content/gdrive/MyDrive/thesis_images/auc/roc_auc_cnn_results.pickle', 'rb') as handle:
  roc_auc_cnn_results = pickle.load(handle)

with open(f'/content/gdrive/MyDrive/thesis_images/auc/roc_auc_vit_results.pickle', 'rb') as handle:
  roc_auc_vit_results = pickle.load(handle)

plt.figure(figsize=(4,3), dpi =1200)
plt.plot(roc_auc_cnn_results[0][0][0], roc_auc_cnn_results[0][0][1], color= 'black', linestyle = '-', label='CNN - RGB (AUC = {:.3f})'.format(roc_auc_cnn_results[0][0][2]))
plt.plot(roc_auc_cnn_results[1][0][0], roc_auc_cnn_results[1][0][1], color= 'black', linestyle = '--', label='CNN - HSV (AUC = {:.3f})'.format(roc_auc_cnn_results[1][0][2]))
plt.plot(roc_auc_cnn_results[2][0][0], roc_auc_cnn_results[2][0][1], color= 'black', linestyle = '-.', label='CNN - YCbCr (AUC = {:.3f})'.format(roc_auc_cnn_results[2][0][2]))
plt.plot(roc_auc_cnn_results[3][0][0], roc_auc_cnn_results[3][0][1], color= 'black', linestyle = ':', label='CNN - LAB (AUC = {:.3f})'.format(roc_auc_cnn_results[3][0][2]))

plt.plot(roc_auc_vit_results[0][0][0], roc_auc_vit_results[0][0][1], color= 'navy', linestyle = '-', label='ViT - RGB (AUC = {:.3f})'.format(roc_auc_vit_results[0][0][2]))
plt.plot(roc_auc_vit_results[1][0][0], roc_auc_vit_results[1][0][1], color= 'navy', linestyle = '--', label='ViT - HSV (AUC = {:.3f})'.format(roc_auc_vit_results[1][0][2]))
plt.plot(roc_auc_vit_results[2][0][0], roc_auc_vit_results[2][0][1], color= 'navy', linestyle = '-.', label='ViT- YCbCr (AUC = {:.3f})'.format(roc_auc_vit_results[2][0][2]))
plt.plot(roc_auc_vit_results[3][0][0], roc_auc_vit_results[3][0][1], color= 'navy', linestyle = ':', label='ViT - LAB (AUC = {:.3f})'.format(roc_auc_vit_results[3][0][2]))


plt.plot([0, 1], [0, 1], color='silver', lw=2, linestyle='-')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc='best')
plt.show()

In [ ]:
#plot ROC curves and AUCs of CNN and Vit - second test set

plt.figure(figsize=(4,3), dpi =1200)

plt.plot(roc_auc_cnn_results[0][1][0], roc_auc_cnn_results[0][1][1], color= 'black', linestyle = '-', label='CNN - RGB (AUC = {:.3f})'.format(roc_auc_vit_results[0][1][2]))
plt.plot(roc_auc_cnn_results[1][1][0], roc_auc_cnn_results[1][1][1], color= 'black', linestyle = '--', label='CNN - HSV (AUC = {:.3f})'.format(roc_auc_vit_results[1][1][2]))
plt.plot(roc_auc_cnn_results[2][1][0], roc_auc_cnn_results[2][1][1], color= 'black', linestyle = '-.', label='CNN - YCbCr (AUC = {:.3f})'.format(roc_auc_vit_results[2][1][2]))
plt.plot(roc_auc_cnn_results[3][1][0], roc_auc_cnn_results[3][1][1], color= 'black', linestyle = ':', label='CNN - LAB (AUC = {:.3f})'.format(roc_auc_vit_results[3][1][2]))

plt.plot(roc_auc_vit_results[0][1][0], roc_auc_vit_results[0][1][1], color= 'navy', linestyle = '-', label='ViT - RGB (AUC = {:.3f})'.format(roc_auc_vit_results[0][1][2]))
plt.plot(roc_auc_vit_results[1][1][0], roc_auc_vit_results[1][1][1], color= 'navy', linestyle = '--', label='ViT - HSV (AUC = {:.3f})'.format(roc_auc_vit_results[1][1][2]))
plt.plot(roc_auc_vit_results[2][1][0], roc_auc_vit_results[2][1][1], color= 'navy', linestyle = '-.', label='ViT - YCbCr (AUC = {:.3f})'.format(roc_auc_vit_results[2][1][2]))
plt.plot(roc_auc_vit_results[3][1][0], roc_auc_vit_results[3][1][1], color= 'navy', linestyle = ':', label='ViT - LAB (AUC = {:.3f})'.format(roc_auc_vit_results[3][1][2]))


plt.plot([0, 1], [0, 1], color='silver', lw=2, linestyle='-')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc='best')
plt.show()

In [ ]:
#calculate average auc
average_auc1 = round((roc_auc_cnn_results[0][0][2] + roc_auc_cnn_results[1][0][2] + roc_auc_cnn_results[2][0][2] + roc_auc_cnn_results[3][0][2] + roc_auc_vit_results[0][0][2] + roc_auc_vit_results[1][0][2] + roc_auc_vit_results[2][0][2] + roc_auc_vit_results[3][0][2])/8,3)
average_auc2 = round((roc_auc_cnn_results[0][1][2] + roc_auc_cnn_results[1][1][2] + roc_auc_cnn_results[2][1][2] + roc_auc_cnn_results[3][1][2] + roc_auc_vit_results[0][1][2] + roc_auc_vit_results[1][1][2] + roc_auc_vit_results[2][1][2] + roc_auc_vit_results[3][1][2])/8,3)

print("auc test 1:", average_auc1)
print("auc test 2:", average_auc2)